In [1]:
import json

# Constantes

In [2]:
# CONSTANTES

TEMPLATE = {
    "id": None,
    "userid": None,
    "origin": None,
    "source_id": "fake90e6d701748f08514b01",
    "source": "fake",
    "pname": None,
    "pvalue": None,
    "context": "application",
    "datapoints": 0,
    "category": None
}

FOLDER = "./data/"
C_USER =  dict(
    datafile ="GAMGameusers.json",
    demographicAtts = ['Gender', 'Age', 'RelationshipWithArt','RelationshipWithMuseum', 'ContentInLIS'],
    userIdAtt = 'authorUsername',
    ugcfile = "ugcUsers.json"
)

C_CONTRIB = dict(
    datafile = "dataset_celi.json",
    userIdAtt = 'authorUsername',
    artifactIdAtt = 'artworkId',
    contribIdAtt = '_id',
    contribAtts = ['itMakesMeFeel','itMakesMeThinkAbout','itRemindsMeOf'],
    contribAttValues = ['emotions', 'sentiment'],
    ugcfile = "ugcContributions.json"
)

_ID = 0


# Usuarios

In [3]:
with open(FOLDER+C_USER["datafile"]) as f:
    userData = json.load(f)


In [4]:
postDict= {}
for user in userData:    
    result = []
    currentData = TEMPLATE.copy()
    currentData["category"] = "demographics"
    currentData["userid"] = user[C_USER["userIdAtt"]]
    for item in C_USER["demographicAtts"]:
        attData = currentData.copy()
        attData["id"] = _ID
        _ID+=1
        attData["origin"] = "User"
        attData["pname"]  = item
        attData["pvalue"] = user[item] 
        result.append(attData)
    postDict[user[C_USER["userIdAtt"]]] = result

In [5]:
with open(FOLDER+C_USER["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)

In [6]:
# postDict es un diccionario en las que las claves son ids de usuarios

In [7]:
# Para hacer los POST:
# server = "http://localhost:8080"
# for key, value in postDict.items():
#    response=requests.post(f'{server}/v1.1/users/{key}/update-generated-content', json = value)
#    procesar el resultado de response

# Contribuciones de los usuarios


In [8]:
with open(FOLDER+C_CONTRIB["datafile"]) as f:
    contribData = json.load(f)
contribData

[{'_id': '6239d59e63c35e2bf9244530',
  'title': "memorie d'estate ",
  'parts': [{'artworkId': '35459',
    'multimediaData': {'text': 'pomeriggi primaverili passati al parco con le mie compagne a studiare  ',
     'emojis': [],
     'tags': [],
     'textTemplate': 'itRemindsMeOf',
     'answersToTemplates': {'itMakesMeFeel': '',
      'itMakesMeThinkAbout': '',
      'itRemindsMeOf': 'pomeriggi primaverili passati al parco con le mie compagne a studiare  '},
     'analysis': {'itMakesMeFeel': {},
      'itMakesMeThinkAbout': {},
      'itRemindsMeOf': {'emotions': {},
       'sentiment': {'Positive': 0.59171665},
       'toxicity': {},
       'entities': {}}}}},
   {'artworkId': '35380',
    'multimediaData': {'text': 'la mia casa in montagna e le giornate trascorse rilassandomi in pineta ',
     'emojis': [],
     'tags': [],
     'textTemplate': 'itRemindsMeOf',
     'answersToTemplates': {'itMakesMeFeel': '',
      'itMakesMeThinkAbout': '',
      'itRemindsMeOf': 'la mia casa in 

In [9]:
def createUserContribution(contrib, contribAtt, contribValue):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        interactionData = currentData.copy()
        interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
        interactionData["pname"] = f'{contribAtt}{sep}{contribValue}'
        analysysValue =  interaction["multimediaData"]["analysis"][contribAtt]
        interactionData["pvalue"] = analysysValue[contribValue] if contribValue in analysysValue else ""
        interactions.append(interactionData)
    return interactions


def createNonEmptyUserContribution(contrib, contribAtt, contribValue):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        analysysValue =  interaction["multimediaData"]["analysis"][contribAtt]
        # Solo añadimos la interacción si hay datos. Si no, ¿para qué la añadimos al CM?
        if contribValue in analysysValue:
            interactionData = currentData.copy()
            interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
            interactionData["pname"] = f'{contribAtt}{sep}{contribValue}'
        
            interactionData["pvalue"] = analysysValue[contribValue]
            interactions.append(interactionData)
    return interactions

In [10]:
def createUserTextContribution(contrib, contribAtt):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        interactionData = currentData.copy()
        interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
        # Add text (e.g. itMakesMeFeel.text)
        interactionData["pname"] = f'{contribAtt}{sep}text'
        textValue =  interaction["multimediaData"]["answersToTemplates"][contribAtt]
        interactionData["pvalue"] = textValue
        interactions.append(interactionData)
    return interactions

def createNonEmptyUserTextContribution(contrib, contribAtt):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        textValue =  interaction["multimediaData"]["answersToTemplates"][contribAtt]
        if len(textValue)>0:
            interactionData = currentData.copy()
            interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
            # Add text (e.g. itMakesMeFeel.text)
            interactionData["pname"] = f'{contribAtt}{sep}text'
            textValue =  interaction["multimediaData"]["answersToTemplates"][contribAtt]
            interactionData["pvalue"] = textValue
            interactions.append(interactionData)
    return interactions

In [11]:
postDict= {}

for contribAtt in C_CONTRIB["contribAtts"]:
    for contribValue in C_CONTRIB["contribAttValues"]:
        key = f'{contribAtt}_{contribValue}'
        attValueDict = {}
        for contrib in contribData: 
            # Si no queremos generar datos para las contribuciones que no aportan nada,
            # sustituir   
            #      createUserContribution
            # con
            #      createNonEmptyUserContribution
            # TODO: ¿Qué pasa si una contribución tiene texto pero no genera emociones? ¿Produce problemas con esto? 
            result = createUserContribution(contrib, contribAtt, contribValue)
            attValueDict[contrib[C_CONTRIB["userIdAtt"]]] = result
        postDict[key] = attValueDict

        # El texto de la contribución es especial porque no tengo que acceder a analysis sino a answersToTemplates
        key = f'{contribAtt}_text'
        attValueDict = {}
        for contrib in contribData: 
            # Si no queremos generar datos para las contribuciones que no aportan nada,
            # sustituir   
            #      createUserTextContribution
            # con
            #      createNonEmptyUserTextContribution
            result = createUserTextContribution(contrib, contribAtt)
            attValueDict[contrib[C_CONTRIB["userIdAtt"]]] = result
        postDict[key] = attValueDict
        
postDict

{'itMakesMeFeel_emotions': {'anqHwMQ0': [{'id': '6239dc746093dc5e107d3090',
    'userid': 'anqHwMQ0',
    'origin': '35227',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '6239dc746093dc5e107d3090',
    'userid': 'anqHwMQ0',
    'origin': '35310',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'}],
  '2LIm9lcH': [{'id': '637ba86a5fe06708aa2277c7',
    'userid': '2LIm9lcH',
    'origin': '35230',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '637ba86a5fe06708aa2277c7',
    'userid': '2LIm9lcH',
    'origin'

In [12]:
# Ejemplo de lo que se genera en cada clave
postDict["itRemindsMeOf_emotions"]

{'anqHwMQ0': [{'id': '6239dc746093dc5e107d3090',
   'userid': 'anqHwMQ0',
   'origin': '35227',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': {'Joy': 1.7947353000000001},
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'},
  {'id': '6239dc746093dc5e107d3090',
   'userid': 'anqHwMQ0',
   'origin': '35310',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': {'Serenity': 1.98994434, 'Joy': 1.0},
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'}],
 '2LIm9lcH': [{'id': '637ba86a5fe06708aa2277c7',
   'userid': '2LIm9lcH',
   'origin': '35230',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': '',
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'},
  {'id': '637ba86a5fe06708aa2277c7',
   'userid': '2LIm9lcH',
   'origin'

In [13]:
with open(FOLDER+C_CONTRIB["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)

In [14]:
# postDict es un diccionario en el que:
# - Las claves son el tipo de interacción que estoy interesado en enviar al servidor. P.e. itRemindsMeOf_text envía contribuciones cuyo pname es itRemindsMeOf.text
# - Los valores son diccionarios en los que las claves son ids de usuarios y los valores son todas las contribuciones de ese usuario y del tipo fijado por la clave anterior. 

# Para hacer los POST:
# server = "http://localhost:8080"
# contribTypes = ["itRemindsMeOf_text", "itRemindsMeOf_emotions"] 
# for ctype in contribTypes:
#    contribsDict = postDict[cType]
#    for key, value in contribsDict.items():
#        response=requests.post(f'{server}/v1.1/users/{key}/update-generated-content', json = value)
        # procesar el resultado de response